In [68]:
import redis
import random
import pandas as pd

In [2]:
r = redis.Redis(
  host='Enter Reddit host here',
  port=16777,
  password='Enter password here',
  decode_responses=True )

# Beer: 20 Get All

In [69]:
# Perform HGETALL operation on the key 'beer:20'
key = 'beer:20'

In [70]:
# Perform HGETALL operation on the key 'beer:20'
beer_details = r.hgetall(key)

In [71]:
beer_details

{'brewery_id': '483',
 'name': 'Cow Palace Scotch Ale 1998',
 'cat_id': '1',
 'style_id': '15',
 'abv': '8.7',
 'ibu': '0.0',
 'srm': '0.0',
 'upc': '0'}

# Get Brewery 119

In [72]:
# Perform HGETALL operation on the key 'beer:20'
key = 'brewery:119'

In [73]:
# Perform HGETALL operation on the key 'beer:20'
brewery_details = r.hgetall(key)

In [74]:
brewery_details

{'name': 'Big Ridge Brewing',
 'address1': '15133 Highway 10',
 'city': 'Surrey',
 'state': 'British Columbia',
 'code': '0',
 'country': 'Canada',
 'phone': '1-604-574-2739'}

# Get a random sample of 1000 beers from a REDIS database

In [9]:
# Pattern for keys representing beers
beer_pattern = 'beer:*'

# Fetch all keys matching the pattern
beer_keys = r.keys(beer_pattern)


In [10]:
# Number of beers to sample
sample_size = 1000

# Sample random keys
random_keys = random.sample(beer_keys, min(len(beer_keys), sample_size))

In [11]:
random_keys[10]

'beer:2975'

In [12]:
def fetch_random_beer_keys(redis_client, sample_size=1000):
    """
    Fetch a random sample of beer keys from Redis.
    """
    cursor = '0'
    sampled_keys = []
    all_keys = []
    
    while cursor != 0:
        cursor, keys = redis_client.scan(cursor=cursor, match="beer:*", count=1000)
        all_keys.extend(keys)
        # Early exit if we've collected enough keys to sample from
        if len(all_keys) >= sample_size * 10:  # Arbitrary multiplier for diversity
            break
    
    # Randomly sample keys if we have more than the desired sample size
    if len(all_keys) > sample_size:
        sampled_keys = random.sample(all_keys, sample_size)
    else:
        sampled_keys = all_keys
    
    return sampled_keys


In [13]:
def fetch_random_beer_keys(redis_client, sample_size=1000):
    """
    Fetch a random sample of beer keys from Redis.
    """
    cursor = '0'
    sampled_keys = []
    all_keys = []
    
    while cursor != 0:
        cursor, keys = redis_client.scan(cursor=cursor, match="beer:*", count=1000)
        all_keys.extend(keys)
    # Randomly sample keys if we have more than the desired sample size
    if len(all_keys) > sample_size:
        sampled_keys = random.sample(all_keys, sample_size)
    else:
        sampled_keys = all_keys
    
    return sampled_keys


In [14]:
def fetch_beer_details(redis_client, keys):
    """
    Fetch details for each beer key.
    """
    beers = [redis_client.hgetall(key) for key in keys]
    return beers

In [15]:
# Fetch 1000 random beer keys
random_beer_keys = fetch_random_beer_keys(r, 1000)

# Fetch details for these beers
random_beers = fetch_beer_details(r, random_beer_keys)

# Load into a pandas DataFrame
df_random_beers = pd.DataFrame(random_beers)


In [75]:
df_random_beers.shape

(1000, 13)

In [16]:
df_random_beers.head()

,cat_id,brewery_id,style_id,upc,name,filepath,abv,ibu,srm,descript,Unnamed:,0,12
0,5,1204,64,0,Vertical Epic 10.10.10,stone-verticalepic101010.jpg,9.5,0.0,0.0,Fermented with the legendary Ardennes strain o...,NaN,NaN,NaN
1,7,4,77,0,Genuine Pilsner,NaN,5.5,0.0,0.0,"The Aass Genuine Pilsner is a premium lager"" p...",11,Unnamed:,7/22/2010 20:00
2,3,723,33,0,La Roja,NaN,7.2,0.0,0.0,An artisan amber ale brewed in the Flanders tr...,NaN,NaN,NaN
3,3,731,33,0,Red Trolley Ale,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,8,1080,95,0,Calumet Wheat,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN


# 100 beers with ABV 7 AND up : ETL Approach

In [18]:
def fetch_first_100_beers_with_abv_7_and_up(redis_client, pattern='beer:*', abv_threshold=7.0, limit=100):
    """
    Fetch the first 500 beer records with ABV of 7 and up from Redis.
    """
    cursor = '0'
    beers_with_high_abv = []
    
    while cursor != 0 and len(beers_with_high_abv) < limit:
        cursor, keys = redis_client.scan(cursor=cursor, match=pattern, count=100)
        for key in keys:
            if len(beers_with_high_abv) >= limit:
                break  # Stop if we've collected enough beers
            beer = redis_client.hgetall(key)
            try:
                abv = float(beer.get('abv', 0))
                if abv >= abv_threshold:
                    beers_with_high_abv.append(beer)
            except ValueError:
                continue  # Skip records where ABV is not a float
    
    return beers_with_high_abv



In [19]:
beers = fetch_first_100_beers_with_abv_7_and_up(r)


In [22]:
beers

[{'brewery_id': '70',
  'name': 'Der Weisse Bock',
  'cat_id': '4',
  'style_id': '55',
  'abv': '8.5',
  'ibu': '0.0',
  'srm': '0.0',
  'upc': '0'},
 {'brewery_id': '1373',
  'name': 'Norfolk Nog Old Dark Ale',
  'cat_id': '1',
  'style_id': '13',
  'abv': '99.99',
  'ibu': '0.0',
  'srm': '0.0',
  'upc': '0'},
 {'ibu': '0.0',
  'srm': '0.0',
  'cat_id': '1',
  'brewery_id': '549',
  'style_id': '15',
  'upc': '0',
  'name': 'Dirty Bastard Scotch Style Ale',
  'abv': '8.5',
  'descript': "Founders flagship beer. Dirty Bastard is an absolute beautiful beer to behold. Dark ruby in color and brewed with ten varieties of imported malts this beer continuously lives up to its reputation as a bold and powerful ale. Dirty Bastard is complex in the finish with hints of smoke and peat paired with a malty richness, finalized with a good bit of hop attitude.  This beer ain't for the wee lads."},
 {'ibu': '0.0',
  'srm': '0.0',
  'cat_id': '4',
  'brewery_id': '960',
  'style_id': '55',
  'upc': 

In [76]:
df_beerswhighabv = pd.DataFrame(beers)

In [77]:
df_beerswhighabv.head()

,brewery_id,name,cat_id,style_id,abv,ibu,srm,upc,descript,Unnamed:,12,Stout,0
0,70,Der Weisse Bock,4,55,8.5,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN
1,1373,Norfolk Nog Old Dark Ale,1,13,99.99,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN
2,549,Dirty Bastard Scotch Style Ale,1,15,8.5,0.0,0.0,0,Founders flagship beer. Dirty Bastard is an ab...,NaN,NaN,NaN,NaN
3,960,Otto's Weizenbock,4,55,7.2,0.0,0.0,0,A strong dark German style wheat bock. This wh...,NaN,NaN,NaN,NaN
4,1298,Northwind Imperial Stout,3,42,7.5,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN


# ELT approach

In [23]:
def fetch_all_beers_and_abvs(redis_client, pattern='beer:*'):
    """
    Fetch all beer keys and their ABV values from Redis.
    """
    cursor = '0'
    beers_and_abvs = []

    while cursor != 0:
        cursor, keys = redis_client.scan(cursor=cursor, match=pattern, count=1000)
        for key in keys:
            abv = redis_client.hget(key, 'abv')
            if abv is not None:
                # Store in a dictionary format for easy DataFrame conversion
                beers_and_abvs.append({'beer_key': key, 'abv': abv})
    
    return beers_and_abvs

In [24]:
all_beers_and_abvs = fetch_all_beers_and_abvs(r)


In [25]:
df_beers = pd.DataFrame(all_beers_and_abvs)

In [26]:
df_beers.head()

,beer_key,abv
0,beer:3265,0.0
1,beer:552,5.0
2,beer:761,0.0
3,beer:1852,0.0
4,beer:1080,0.0


In [28]:
df_beers['abv'] = df_beers['abv'].astype(float)


In [29]:
df_filtered = df_beers[df_beers['abv'] > 7]


In [30]:
df_filtered.shape

(753, 2)

In [50]:
df_filtered.head()

,beer_key,abv
5,beer:211,8.50
10,beer:1859,99.99
36,beer:1013,8.50
40,beer:4756,7.20
45,beer:1335,7.50


# ELT Pipeline for all

In [31]:
def fetch_all_matching_data(redis_client, pattern):
    """
    Fetch all data for keys that match the given pattern.
    """
    cursor = '0'
    matching_data = []

    # Create a pipeline for efficient batch operations
    pipeline = redis_client.pipeline()

    while cursor != 0:
        cursor, keys = redis_client.scan(cursor=cursor, match=pattern, count=1000)
        for key in keys:
            # Queue up HGETALL operations for each key in the pipeline
            pipeline.hgetall(key)
        
        # Execute the pipeline and extend our list with the results
        matching_data.extend(pipeline.execute())

    return matching_data


In [32]:

# Example usage: Fetch all data for keys that match the 'brewery:*' pattern
pattern = 'brewery:*'
all_brewery_data = fetch_all_matching_data(r, pattern)


In [33]:
df_breweries = pd.DataFrame(all_brewery_data)


In [34]:
df_breweries.shape

(1406, 11)

In [35]:
df_breweries.head()

,country,state,name,city,address1,code,website,phone,descript,filepath,address2
0,Germany,Sachsen,Brauhaus am WaldschlÃ¶sschen,Dresden,Am Brauhaus 8b,1099,http://www.waldschloesschen.de,+49 351 - 652 39 00,"traditional brew house, founded 1836, make the...",NaN,NaN
1,United States,Idaho,Grand Teton Brewing #2,Victor,430 Old Jackson Highway,83455,http://www.grandtetonbrewing.com/,1-208-787-9000,NaN,NaN,NaN
2,United States,MO,The St. Louis Brewrey,St. Louis,2100 Locust Street,63103,http://http://www.schlafly.com,314-241-2337,The Schlafly Tap Room first opened its doors i...,schlafly.3color_.jpg,NaN
3,Belgium,Hainaut,Brasserie de Brunehaut,Brunehaut,Rue de Panneries 17,7623,http://www.brunehaut.com/,32-069-34-64-11,"An artisanal (non-conglomerate) brewer, Bruneh...",NaN,NaN
4,Germany,Bayern,Hofmark Brauerei,Cham,Hofmarkstrae 15,NaN,NaN,49-(0)9971-/-3301,NaN,NaN,NaN


# Get all Beer data

In [36]:
def fetch_all_beer_data(redis_client, pattern='beer:*'):
    """
    Fetch all beer data from Redis, including the key as 'id'.
    """
    cursor = '0'
    all_beers = []
    pipeline = redis_client.pipeline()

    while cursor != 0:
        cursor, keys = redis_client.scan(cursor=cursor, match=pattern, count=1000)
        for key in keys:
            pipeline.hgetall(key)  # Queue the HGETALL command in the pipeline
            
        # Execute all commands in the pipeline and store responses
        responses = pipeline.execute()
        
        # Combine key ('id') with its data for each beer
        for key, beer_data in zip(keys, responses):
            beer_data['id'] = key  # Include the key as 'id'
            all_beers.append(beer_data)
    
    return all_beers

In [37]:
# Fetch all beer data, including the key as 'id'
all_beer_data = fetch_all_beer_data(r)


In [38]:

# Convert the list of dictionaries into a pandas DataFrame
df_beers = pd.DataFrame(all_beer_data)

In [39]:
df_beers.head()


,brewery_id,name,cat_id,style_id,abv,ibu,srm,upc,id,descript,Unnamed:,0,12,Stout,filepath
0,582,Hibernator Winter Ale,-1,-1,0.0,0.0,0.0,0,beer:3265,NaN,NaN,NaN,NaN,NaN,NaN
1,1116,Weizen,4,50,5.0,0.0,0.0,0,beer:552,NaN,NaN,NaN,NaN,NaN,NaN
2,526,Light,-1,-1,0.0,0.0,0.0,0,beer:761,NaN,NaN,NaN,NaN,NaN,NaN
3,636,Altruistic American Ale,3,26,0.0,0.0,0.0,0,beer:1852,NaN,NaN,NaN,NaN,NaN,NaN
4,693,Mojave Red,8,95,0.0,0.0,0.0,0,beer:1080,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df_beers.shape


(5213, 15)

# Get data for all beers from breweries in Belgium in database 

In [41]:

def fetch_belgian_brewery_ids(redis_client):
    """Fetch IDs of breweries located in Belgium."""
    belgian_brewery_ids = []
    cursor = '0'
    while cursor != 0:
        cursor, keys = redis_client.scan(cursor=cursor, match="brewery:*", count=1000)
        for key in keys:
            country = redis_client.hget(key, 'country')
            if country == 'United States':
                # Extract and store the brewery ID from the key
                brewery_id = key.split(":")[1]
                belgian_brewery_ids.append(brewery_id)
    return belgian_brewery_ids



In [42]:

# Step 1: Fetch Belgian brewery IDs
belgian_brewery_ids = fetch_belgian_brewery_ids(r)


In [43]:
belgian_brewery_ids

['594',
 '1404',
 '1256',
 '122',
 '138',
 '1095',
 '155',
 '1183',
 '868',
 '159',
 '257',
 '575',
 '916',
 '1264',
 '615',
 '57',
 '582',
 '352',
 '1238',
 '1128',
 '1008',
 '441',
 '318',
 '39',
 '535',
 '151',
 '423',
 '855',
 '725',
 '965',
 '815',
 '160',
 '978',
 '1002',
 '132',
 '1279',
 '641',
 '476',
 '47',
 '1293',
 '107',
 '1177',
 '69',
 '56',
 '471',
 '891',
 '55',
 '1235',
 '500',
 '949',
 '602',
 '716',
 '1284',
 '1139',
 '114',
 '1059',
 '841',
 '922',
 '984',
 '731',
 '377',
 '899',
 '1164',
 '673',
 '16',
 '587',
 '437',
 '1248',
 '809',
 '142',
 '1294',
 '821',
 '450',
 '478',
 '407',
 '834',
 '952',
 '337',
 '144',
 '767',
 '444',
 '133',
 '655',
 '501',
 '456',
 '1200',
 '1029',
 '860',
 '1254',
 '780',
 '1144',
 '468',
 '560',
 '969',
 '1309',
 '830',
 '1338',
 '882',
 '1332',
 '817',
 '935',
 '1105',
 '1276',
 '562',
 '1418',
 '1190',
 '848',
 '405',
 '464',
 '524',
 '1055',
 '427',
 '588',
 '1100',
 '565',
 '59',
 '1249',
 '1283',
 '421',
 '314',
 '1244',
 '358

In [51]:
def fetch_beers_from_breweries(redis_client, brewery_ids):
    """Fetch all beers from the given list of brewery IDs."""
    all_beers = []
    for brewery_id in brewery_ids:  # iterate through brewery IDs
        cursor = b'0'  # initialize cursor properly
        while cursor != 0:
            cursor, keys = redis_client.scan(cursor=cursor, match=f"beer:*", count=1000)
            for key in keys:
                beer_data = redis_client.hgetall(key)
                if 'brewery_id' in beer_data and beer_data['brewery_id'] == brewery_id:
                    all_beers.append(beer_data)
            cursor = int(cursor)  # convert cursor to integer for comparison
    return all_beers


In [57]:
# Load the beer data into a pandas DataFrame
df_belgian_beers = pd.DataFrame(belgian_beers)


In [58]:
df_belgian_beers.head()

""


In [64]:
def test_belgian_brewery(redis_client):
    country = redis_client.hget("brewery:3", "country")
    print(f"Brewery 3 country: {country}")

# Use this function to test the retrieval of the country for brewery:3
test_belgian_brewery(r)


Brewery 3 country: Belgium


In [105]:

# Fetch and print the country for brewery:3
country = r.hget("brewery:3", "country")

In [106]:
country

'Belgium'